In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib
import functools
import itertools
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from functools import reduce
from networkx.algorithms import community
pd.show_versions()

C:\Anaconda\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(



INSTALLED VERSIONS
------------------
commit           : b5958ee1999e9aead1938c0bba2b674378807b3d
python           : 3.8.3.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19041
machine          : AMD64
processor        : Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : Dutch_Belgium.1252

pandas           : 1.1.5
numpy            : 1.18.5
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.1.1
setuptools       : 49.2.0.post20200714
Cython           : 0.29.14
pytest           : 5.4.3
hypothesis       : None
sphinx           : 3.1.2
blosc            : None
feather          : None
xlsxwriter       : 1.2.9
lxml.etree       : 4.5.2
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.16.1
pandas_datareader: None
bs4              : 4.9.1
bottleneck       : 1.

# PPI data

## STRING Data

In [2]:
#Get the data and create a STRING dataframe
string1 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\9606.protein.links.full.v11.0.txt.gz', compression='gzip', header=0, sep=' ')
string1 = pd.DataFrame(string1)
string = string1.copy()

In [3]:
print(f'The amount of original inetractions: {string.shape[0]}')
print(f'The amount of original proteins: {len(set(string.protein1.tolist() + string.protein2.tolist()))}')

The amount of original inetractions: 11759454
The amount of original proteins: 19354


In [3]:
# Only retain the most necessary columns and 
# create a column named 'Database' to know which database contained which interaction
string = string[['protein1', 'protein2', 'combined_score']]
string['Database_S'] = 'S'

# Remove all duplicated protein pairs
string['sorted_row'] = [sorted([a,b]) for a,b in zip(string.protein1, string.protein2)]
string['sorted_row'] = string['sorted_row'].astype(str)
string.drop_duplicates(subset=['sorted_row'], inplace=True)
string = string.drop(columns = ['sorted_row'])

### Download all the uniprot ID's and match them with the STRING protein ID's

### Get all the STRING protein ID's

In [4]:
# Protein 1 column
names1 = string['protein1'].unique()
string_names_protein1 = names1.tolist()
f = open('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\string_names_protein1.txt','w')
f.write('\n'.join(string_names_protein1))
f.close()

# Protein 2 column
names2 = string['protein2'].unique()
string_names_protein2 = names2.tolist()
f = open('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\string_names_protein2.txt','w')
f.write('\n'.join(string_names_protein2))
f.close()

In [5]:
# Protein 1 column
uniprot_string_protein1 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot-protein1.tab.gz', compression='gzip', sep = '\t')
uniprot_string_protein1.rename(columns = {'yourlist:M20201112E5A08BB0B2D1C45B0C7BC3B55FD2655603E3E1X':'protein', 
                                          'Entry':'UniProtKB_ID'}, inplace=True)
uniprot_string_protein1 = uniprot_string_protein1[['protein', 'UniProtKB_ID']]

# Protein 2 column
uniprot_string_protein2 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot-protein2.tab.gz', compression='gzip', sep = '\t')
uniprot_string_protein2.rename(columns = {'yourlist:M20201112216DA2B77BFBD2E6699CA9B6D1C41EB203E4D3V':'protein', 
                                          'Entry':'UniProtKB_ID'}, inplace=True)
uniprot_string_protein2 = uniprot_string_protein2[['protein', 'UniProtKB_ID']]

# Combine into one dataframe
uniprot_string = pd.concat([uniprot_string_protein1, uniprot_string_protein2])
uniprot_string = uniprot_string.drop_duplicates()

In [6]:
uniprot_string.shape

(18501, 2)

In [7]:
uniprot_string.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\US_IDs.csv', index = False, sep = '\t')

### Combine the UniProt ID's with the STRING dataframe

In [8]:
# Merge the Uniprot IDs together with the STRING IDs
string_uniprot1 = pd.merge(string, uniprot_string, left_on='protein1', right_on='protein')
string_uniprot2 = pd.merge(string_uniprot1, uniprot_string, left_on='protein2', right_on='protein')
string_uniprot2 = string_uniprot2[['UniProtKB_ID_x', 
                                   'UniProtKB_ID_y', 
                                   'protein_x', 
                                   'protein_y', 
                                   'Database_S', 
                                   'combined_score']]

# Rename the columns
string_uniprot2.rename(columns = {'UniProtKB_ID_x' : 'UniprotAccessionA', 
                                  'UniProtKB_ID_y' : 'UniprotAccessionB', 
                                  'protein_x' : 'STRING_IDA',
                                  'protein_y' : 'STRING_IDB', 
                                  'combined_score' : 'STRING_score'}, inplace=True)

In [9]:
# Sort the IDs so later te merging will happen on the same interaction pairs and create a new dataframe out of it
ids = [sorted([a,b]) for a,b in zip(string_uniprot2.UniprotAccessionA, string_uniprot2.UniprotAccessionB)]
d = {'UniprotAccessionA' : [ids[i][0] for i in range(len(ids))], 
     'UniprotAccessionB' : [ids[i][1] for i in range(len(ids))], 
     'Database_S' : string_uniprot2.Database_S, 
     'STRING_score': string_uniprot2.STRING_score}
string_uniprot3 = pd.DataFrame(data=d)
string_uniprot4 = pd.merge(string_uniprot3, uniprot_string, left_on='UniprotAccessionA', right_on='UniProtKB_ID')
string_uniprot = pd.merge(string_uniprot4, uniprot_string, left_on='UniprotAccessionB', right_on='UniProtKB_ID')

# Retain the correct columns
string_uniprot = string_uniprot[['UniprotAccessionA', 
                                 'UniprotAccessionB', 
                                 'protein_x', 
                                 'protein_y', 
                                 'Database_S', 
                                 'STRING_score']]

# Rename the columns
string_uniprot.rename(columns = {'protein_x' : 'STRING_IDA',
                                 'protein_y' : 'STRING_IDB'}, inplace=True)

In [10]:
string_uniprot.shape

(5538412, 6)

In [11]:
string_uniprot.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\STRING_UniProt.csv', index = False, sep = '\t')

## BioGRID Data

In [9]:
#Get the data and create a BioGRID dataframe
biogrid1 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\BIOGRID-ORGANISM-Homo_sapiens-3.5.188.tab3.txt', delimiter='\t', low_memory = False)
biogrid1 = pd.DataFrame(biogrid1)
biogrid = biogrid1.copy()

In [25]:
print(f'The amount of original inetractions: {biogrid.shape[0]}')
x = len(set(list(biogrid['BioGRID ID Interactor A']) + list(biogrid['BioGRID ID Interactor B'])))
print(f'The amount of original proteins: {x}')

The amount of original inetractions: 616492
The amount of original proteins: 25772


In [13]:
# Only retain the most necessary columns and 
# create a column named 'Database' to know which database contained which interaction
biogrid = biogrid[['BioGRID ID Interactor A', 
                   'BioGRID ID Interactor B', 
                   'Score', 
                   'Ontology Term IDs']]
biogrid['Database_B'] = 'B'

# Turn the type of the columns to strings
biogrid = biogrid.astype(str)

### Download all the uniprot ID's and match them with the BioGRID protein ID's

### Get all the BioGRID protein ID's

In [14]:
# BioGRID ID Interactor A column
names11 = biogrid['BioGRID ID Interactor A'].unique()
biogrid_names_interactora = names11.tolist()
f = open('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\biogrid_names_interactorA.txt','w')
f.write('\n'.join(biogrid_names_interactora))
f.close()

# BioGRID ID Interactor B column
names22 = biogrid['BioGRID ID Interactor B'].unique()
biogrid_names_interactorb = names22.tolist()
f = open('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\biogrid_names_interactorB.txt','w')
f.write('\n'.join(biogrid_names_interactorb))
f.close()

In [15]:
# BioGRID ID Interactor A column
uniprot_biogrid_interactora = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot-interactorA.tab.gz', compression='gzip', sep = '\t')
uniprot_biogrid_interactora.rename(columns = {'yourlist:M202011128471C63D39733769F8E060B506551E120428BAB':'BioGRID_ID', 
                                              'Entry':'UniProtKB_ID'}, inplace=True)
uniprot_biogrid_interactora = uniprot_biogrid_interactora.loc[:, ['BioGRID_ID', 
                                                                  'UniProtKB_ID']]

# BioGRID ID Interactor B column
uniprot_biogrid_interactorb = pd.read_csv(r'D:\\Jana De Coster\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot-interactorB.tab.gz', compression='gzip', sep = '\t')
uniprot_biogrid_interactorb.rename(columns = {'yourlist:M202011128471C63D39733769F8E060B506551E120428AF2':'BioGRID_ID', 
                                              'Entry':'UniProtKB_ID'}, inplace=True)
uniprot_biogrid_interactorb = uniprot_biogrid_interactorb.loc[:, ['BioGRID_ID', 
                                                                  'UniProtKB_ID']]

In [16]:
# Combine into one dataframe
uniprot_biogrid = pd.concat([uniprot_biogrid_interactora, uniprot_biogrid_interactorb])
uniprot_biogrid = uniprot_biogrid.drop_duplicates()
uniprot_biogrid.shape

(17222, 2)

In [17]:
uniprot_biogrid.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\UB_IDs.csv', index = False, sep = '\t')

### Combine the UniProt ID's with the BioGRID dataframe

In [18]:
# Merge the Uniprot IDs together with the BioGRID IDs
biogrid_uniprot1 = pd.merge(biogrid, uniprot_biogrid, left_on = 'BioGRID ID Interactor A', right_on = 'BioGRID_ID')
biogrid_uniprot2 = pd.merge(biogrid_uniprot1, uniprot_biogrid, left_on = 'BioGRID ID Interactor B', right_on = 'BioGRID_ID')
biogrid_uniprot2 = biogrid_uniprot2[['UniProtKB_ID_x', 
                                     'UniProtKB_ID_y', 
                                     'BioGRID_ID_x', 
                                     'BioGRID_ID_y', 
                                     'Database_B', 
                                     'Score', 
                                     'Ontology Term IDs']]

# Rename the columns
biogrid_uniprot2.rename(columns = {'UniProtKB_ID_x' : 'UniprotAccessionA', 
                                   'UniProtKB_ID_y' : 'UniprotAccessionB', 
                                   'BioGRID_ID_x' : 'BioGRID_IDA',
                                   'BioGRID_ID_y' : 'BioGRID_IDB', 
                                   'Score' : 'BioGRID_score', 
                                   'Ontology Term IDs' : 'BioGRID_ontology_ID'}, inplace=True)

In [19]:
# Sort the IDs so later te merging will happen on the same interaction pairs and create a new dataframe out of it
ids = [sorted([a,b]) for a,b in zip(biogrid_uniprot2.UniprotAccessionA, biogrid_uniprot2.UniprotAccessionB)]
d = {'UniprotAccessionA' : [ids[i][0] for i in range(len(ids))], 
     'UniprotAccessionB' : [ids[i][1] for i in range(len(ids))], 
     'Database_B' : biogrid_uniprot2.Database_B, 
     'BioGRID_score' : biogrid_uniprot2.BioGRID_score, 
     'BioGRID_ontology_ID' : biogrid_uniprot2.BioGRID_ontology_ID}
biogrid_uniprot3 = pd.DataFrame(data=d)
biogrid_uniprot4 = pd.merge(biogrid_uniprot3, uniprot_biogrid, left_on='UniprotAccessionA', right_on='UniProtKB_ID')
biogrid_uniprot = pd.merge(biogrid_uniprot4, uniprot_biogrid, left_on='UniprotAccessionB', right_on='UniProtKB_ID')

# Retain the correct columns
biogrid_uniprot = biogrid_uniprot[['UniprotAccessionA', 
                                   'UniprotAccessionB', 
                                   'BioGRID_ID_x', 
                                   'BioGRID_ID_y', 
                                   'Database_B', 
                                   'BioGRID_score', 
                                   'BioGRID_ontology_ID']]

# Rename the columns
biogrid_uniprot.rename(columns = {'BioGRID_ID_x' : 'BioGRID_IDA',
                                  'BioGRID_ID_y' : 'BioGRID_IDB'}, inplace=True)

In [20]:
biogrid_uniprot.shape

(560450, 7)

In [21]:
biogrid_uniprot.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\BioGRID_UniProt.csv', index = False, sep = '\t')

## IntAct Data

In [4]:
#Get the data and create a IntAct dataframe
intact1 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\species__H.txt', delimiter='\t')
intact1 = pd.DataFrame(intact1)
intact = intact1.copy()

In [6]:
print(f'The amount of original inetractions: {intact.shape[0]}')
x = len(set(list(intact['#ID(s) interactor A']) + list(intact['ID(s) interactor B'])))
print(f'The amount of original proteins: {x}')

The amount of original inetractions: 628794
The amount of original proteins: 41810


In [23]:
# Only retain the most necessary columns and 
# create a column named 'Database' to know which database contained which interaction
intact = intact[['#ID(s) interactor A',
                 'ID(s) interactor B', 
                 'Confidence value(s)']]
intact['Database_I'] = 'I'

#Rename the columns
intact.rename(columns = {'#ID(s) interactor A' : 'UniprotAccessionA',
                         'ID(s) interactor B' : 'UniprotAccessionB', 
                         'Confidence value(s)' : 'IntAct_score'}, inplace=True)

# Remove prefixes in the columns 'UniprotAccessionA', 'UniprotAccessionB' and 'IntAct_score'
intact['UniprotAccessionA'] = intact['UniprotAccessionA'].str.replace('uniprotkb:', '')
intact['UniprotAccessionB'] = intact['UniprotAccessionB'].str.replace('uniprotkb:', '')
intact['IntAct_score'] = intact['IntAct_score'].str.replace('intact-miscore:', '')

# Make the IntAct score a numeric value
intact['IntAct_score'] = pd.to_numeric(intact['IntAct_score'], errors='coerce')

# Only retain the ID's that are reviewed in UniProt
uniprot1 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot_homo_sapiens_reviewed.tab.gz', compression='gzip', sep = '\t')
uniprot = uniprot1['Entry'].tolist()
intact = intact[intact.UniprotAccessionA.isin(uniprot)]
intact = intact[intact.UniprotAccessionB.isin(uniprot)]

In [24]:
# Sort the IDs so later te merging will happen on the same interaction pairs and create a new dataframe out of it
ids = [sorted([a,b]) for a,b in zip(intact.UniprotAccessionA, intact.UniprotAccessionB)]
d = {'UniprotAccessionA' : [ids[i][0] for i in range(len(ids))], 
     'UniprotAccessionB' : [ids[i][1] for i in range(len(ids))], 
     'Database_I' : intact.Database_I, 
     'IntAct_score' : intact.IntAct_score}
intact_uniprot = pd.DataFrame(data=d)

In [25]:
intact_uniprot.shape

(394598, 4)

In [26]:
intact_uniprot.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\IntAct_UniProt.csv', index = False, sep = '\t')

# Combine all the PPI dataframes together

In [27]:
# Merge the three dataframes together based on the UniProt IDs an dremove the rows that contain the same protein twice
sb = pd.merge(string_uniprot, biogrid_uniprot, how = 'outer', on = ['UniprotAccessionA', 'UniprotAccessionB'])
ppi_all = pd.merge(sb, intact_uniprot, how = 'outer', on = ['UniprotAccessionA', 'UniprotAccessionB'])
ppi_all = ppi_all.loc[ppi_all['UniprotAccessionA'] != ppi_all['UniprotAccessionB']]
ppi_all.shape

(6584525, 13)

In [28]:
# Combine the database origin into one column
ppi_all['Database_S'] = ppi_all['Database_S'].fillna('')
ppi_all['Database_B'] = ppi_all['Database_B'].fillna('')
ppi_all['Database_I'] = ppi_all['Database_I'].fillna('')
ppi_all['Database'] = ppi_all['Database_I'] + ppi_all['Database_B'] + ppi_all['Database_S']

# Readjust the columns
ppi_all = ppi_all[['UniprotAccessionA', 
                   'UniprotAccessionB', 
                   'BioGRID_IDA', 
                   'BioGRID_IDB', 
                   'STRING_IDA', 
                   'STRING_IDB',
                   'Database', 
                   'IntAct_score', 
                   'BioGRID_score', 
                   'STRING_score', 
                   'BioGRID_ontology_ID']]
ppi_all.shape

(6584525, 11)

In [29]:
# Save the dataframe
ppi_all.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_data.csv', index = False, sep = '\t')

# Filter the PPI dataframe

In [30]:
ppi_all = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_data.csv', low_memory = False, sep = '\t')

In [31]:
# First adjust the STRING score to get a value between 0 and 1
ppi_all['STRING_score'] = ppi_all['STRING_score'] / 1000

# Filter the PPI data based on preset values:
# STRING scores should be above 0. OR Intact scores should be above 0.4
# BioGRID scores are filtered based on having a confidence score since these scores are derived based on different methods

ppi = ppi_all[((ppi_all['STRING_score'] >= 0.55) 
              & (ppi_all['BioGRID_score'] != '-'))
              | ((ppi_all['IntAct_score'] >= 0.40)
              & (ppi_all['BioGRID_score'] != '-'))
              | ((ppi_all['IntAct_score'] >= 0.40)
              & (ppi_all['STRING_score'] >= 0.55)
              & (ppi_all['BioGRID_score'] != '-'))]

In [32]:
ppi.shape

(611761, 11)

In [33]:
ppi['Database'].value_counts()

S      494765
IBS     61480
BS      20903
IS      20379
I       12055
IB       2179
Name: Database, dtype: int64

In [34]:
ppi.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_data_filtered.csv', index = False, sep = '\t')

# The metadata

## GO Data

In [35]:
go_data = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot_homo_sapiens_reviewed.tab.gz', compression='gzip', sep = '\t')

# Retain only the necessary columns and change their names
go_data = go_data[['Entry', 
                   'Protein names', 
                   'Gene names', 
                   'Gene ontology (biological process)', 
                   'Gene ontology (cellular component)', 
                   'Gene ontology (molecular function)']]
go_data.rename(columns = {'Entry' : 'UniprotAccession',
                          'Protein names' : 'Protein_name', 
                          'Gene names' : 'Gene_names', 
                          'Gene ontology (biological process)' : 'GO_biological_process', 
                          'Gene ontology (cellular component)' : 'GO_cellular_component', 
                          'Gene ontology (molecular function)' : 'GO_molecular_function'}, inplace = True)

# Save the dataframe
go_data.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\GO_data.csv', index = False, sep = '\t')

## Reactome Data

In [36]:
reactome_data = pd.read_table("https://reactome.org/download/current/UniProt2Reactome_All_Levels.txt", 
                              sep = '\t', names = ['UniProtID',
                                                   'Reactome Pathway Stable identifier',
                                                   'URL',
                                                   'Event (Pathway or Reaction) Name',
                                                   'Evidence Code',
                                                   'Species'])
# Select only human related pathways
reactome_data = reactome_data[reactome_data['Species'] == 'Homo sapiens']

# Retain only the necessary columns and change their names
reactome_data = reactome_data[['UniProtID', 
                               'Reactome Pathway Stable identifier', 
                               'Event (Pathway or Reaction) Name']]
reactome_data.rename(columns = {'UniProtID' : 'UniprotAccession', 
                                'Reactome Pathway Stable identifier' : 'Reactome_ID',
                                'Event (Pathway or Reaction) Name' : 'Reactome_name'}, inplace = True)

# Organize per UniProt ID
reactome_data['Reactome_ID'] = reactome_data.groupby(['UniprotAccession'])['Reactome_ID']\
                                            .transform(lambda x: '; '.join(x))
reactome_data['Reactome_name'] = reactome_data.groupby(['UniprotAccession'])['Reactome_name']\
                                              .transform(lambda x: '; '.join(x))


# Save the dataframe
reactome_data.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\Reactome_data.csv', index = False, sep = '\t')

## Human Protein Atlas Data

In [37]:
human_protein_atlas_data = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\proteinatlas.tsv', sep='\t')

# Retain only the necessary columns and change their names
human_protein_atlas_data = human_protein_atlas_data[['Uniprot', 
                                                     'Gene', 
                                                     'Gene description', 
                                                     'Subcellular location']]
human_protein_atlas_data.rename(columns = {'Uniprot' : 'UniprotAccession',
                                           'Gene' : 'HPA_gene', 
                                           'Gene description' : 'HPA_gene_description', 
                                           'Subcellular location' : 'HPA_Subcellular_location'}, inplace = True)

# Save the dataframe
human_protein_atlas_data.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\Human_Protein_Atlas_data.csv', index = False, sep = '\t')

## CORUM Data

In [38]:
corum_data = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\CORUM_allComplexes.txt', delimiter='\t', low_memory = False)

# Select only human related pathways
corum_data = corum_data[corum_data['Organism'] == 'Human']

# Retain only the necessary columns and change their names
corum_data = corum_data[['ComplexName', 
                         'subunits(UniProt IDs)']]

corum_data.rename(columns = {'ComplexName' : 'CORUM_complexes',
                             'subunits(UniProt IDs)' : 'CORUM_subunit_IDs'}, inplace = True)

# Organize per UniProt ID
corum_data = corum_data.assign(UniprotAccession = corum_data['CORUM_subunit_IDs'].str.split(';')).explode('UniprotAccession')
corum_data['ComplexName'] = corum_data.groupby(['UniprotAccession'])['CORUM_complexes']\
                                      .transform(lambda x: '; '.join(x))
corum_data['CORUM_subunit_IDs'] = corum_data.groupby(['UniprotAccession'])['CORUM_subunit_IDs']\
                                            .transform(lambda x: '; '.join(x))

# reset the index and rearrange the columns
corum_data.reset_index(inplace = True)
corum_data = corum_data[['UniprotAccession',
                         'CORUM_complexes',
                         'CORUM_subunit_IDs']]

# Save the dataframe
corum_data.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\CORUM_data.csv', sep = '\t')

## DisGeNet Data

In [8]:
disgenet_or_data = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\DisGeNet_all_gene_disease_associations.tsv.gz', compression = 'gzip', sep='\t')
disgenet_uniprot = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\DisGeNet_uniprot_mapping.tsv.gz', compression = 'gzip', sep='\t')

# Merge the UniProt IDs on the DisGeNet data
disgenet_data = pd.merge(disgenet_or_data, disgenet_uniprot, left_on='geneId', right_on='GENEID')

# Retain only the necessary columns and change their names
disgenet_data = disgenet_data[['UniProtKB',  
                               'diseaseName', 
                               'diseaseId',
                               'diseaseSemanticType', 
                               'geneSymbol', 
                               'score']]

disgenet_data.rename(columns = {'UniProtKB' : 'UniprotAccession',
                                'diseaseName' : 'DisGeNet_disease_name', 
                                'diseaseId' : 'DisGeNet_disease_ID', 
                                'diseaseSemanticType' : 'DisGeNet_class', 
                                'geneSymbol' : 'DisGeNet_gene', 
                                'score' : 'DisGeNet_score'}, inplace = True)
print(f'The amount of DisGeNet disease-associated proteins prior to filtration is : {disgenet_data.shape[0]}')

# Filter the DisGeNet data
disgenet_data = disgenet_data[disgenet_data['DisGeNet_score'] >= 0.35]
print(f'The amount of DisGeNet disease-associated proteins post-filtration is : {disgenet_data.shape[0]}')

# Organize per UniProt ID 
disgenet_data['DisGeNet_score'] = disgenet_data['DisGeNet_score'].astype(str)
disgenet_data['DisGeNet_disease_name'] = disgenet_data.groupby(['UniprotAccession'])['DisGeNet_disease_name']\
                                         .transform(lambda x: '; '.join(x))

disgenet_data['DisGeNet_disease_ID'] = disgenet_data.groupby(['UniprotAccession'])['DisGeNet_disease_ID']\
                                       .transform(lambda x: '; '.join(x))
disgenet_data['DisGeNet_class'] = disgenet_data.groupby(['UniprotAccession'])['DisGeNet_class']\
                                                                  .transform(lambda x: '; '.join(x))
disgenet_data['DisGeNet_gene'] = disgenet_data.groupby(['UniprotAccession'])['DisGeNet_gene']\
                                                                  .transform(lambda x: '; '.join(x))
disgenet_data['DisGeNet_score'] = disgenet_data.groupby(['UniprotAccession'])['DisGeNet_score']\
                                                                   .transform(lambda x: '; '.join(x))

disgenet_data.drop_duplicates(inplace = True)
disgenet_data.reset_index(drop = True, inplace = True)

# Save the dataframe
disgenet_data.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\DisGeNet_data.csv', index = False, sep = '\t')

The amount of DisGeNet disease-associated proteins prior to filtration is : 1057185
The amount of DisGeNet disease-associated proteins post-filtration is : 20594


## DisProt Data

In [41]:
disprot_data = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\DisProt_data.tsv', sep='\t')

# Select only human related pathways
disprot_data = disprot_data[disprot_data['organism'] == 'Homo sapiens']

# Retain only the necessary columns and change their names
disprot_data = disprot_data[['acc',  
                             'name', 
                             'disprot_id']]

disprot_data.rename(columns = {'acc' : 'UniprotAccession', 
                               'name' : 'DisProt_name', 
                               'disprot_id' : 'DisProt_ID'}, inplace = True)

# Remove all duplicates and reset the index
disprot_data.drop_duplicates(inplace = True)
disprot_data.reset_index(inplace = True)
disprot_data.drop(columns = {'index'}, inplace = True)

# Save the dataframe
disprot_data.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\DisProt_data.csv', index = False, sep = '\t')

# Combine all the metadata into 1 dataframe

In [42]:
reactome_data = reactome_data.drop_duplicates('UniprotAccession')
g_r = pd.merge(go_data, reactome_data, on = 'UniprotAccession', how='outer')

human_protein_atlas_data = human_protein_atlas_data.drop_duplicates('UniprotAccession')
g_r_h = pd.merge(g_r, human_protein_atlas_data, on = 'UniprotAccession', how='outer')

corum_data = corum_data.drop_duplicates('UniprotAccession')
g_r_h_c = pd.merge(g_r_h, corum_data, on = 'UniprotAccession', how='outer')

disgenet_data = disgenet_data.drop_duplicates('UniprotAccession')
g_r_h_c_d = pd.merge(g_r_h_c, disgenet_data, on = 'UniprotAccession', how='outer')

disprot_data = disprot_data.drop_duplicates('UniprotAccession')
metadata = pd.merge(g_r_h_c_d, disprot_data, on = 'UniprotAccession', how='outer')

# Select only the UniProt IDs that are reviewed in UniProt and remove all duplicates
uniprot1 = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\UniProt IDs 20_08_2020\\uniprot_homo_sapiens_reviewed.tab.gz', compression='gzip', sep = '\t')
uniprot = uniprot1['Entry'].tolist()
metadata = metadata[metadata.UniprotAccession.isin(uniprot)]
metadata.drop_duplicates(inplace = True)

# Save the dataframe
metadata.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\Metadata.csv', index = False, compression = 'gzip', sep = '\t')

# Combine the metadata and the PPI data together

In [43]:
ppi = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_data_filtered.csv', low_memory = False, sep='\t')
metadata = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\Metadata.csv', low_memory = False, compression = 'gzip', sep = '\t')

In [44]:
# Merge the metadata together with the PPI data
ppi_A = pd.merge(ppi, metadata, how = 'left', left_on = ['UniprotAccessionA'], right_on = ['UniprotAccession'])
ppi_meta = pd.merge(ppi_A, metadata, how = 'left', left_on = ['UniprotAccessionB'], right_on = ['UniprotAccession'])

# Change the columns suffixes
ppi_meta.columns = ppi_meta.columns.str.replace(r'_x', 'A')
ppi_meta.columns = ppi_meta.columns.str.replace(r'_y', 'B')

#Remove duplicate columns
ppi_meta = ppi_meta.loc[:,~ppi_meta.columns.duplicated()]

#Save the dataframe both in csv and txt format
ppi_meta.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_Metadata_IDs.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
ppi_meta.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_Metadata_IDs.txt', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

In [45]:
ppi_meta.head()

,UniprotAccessionA,UniprotAccessionB,BioGRID_IDA,BioGRID_IDB,STRING_IDA,STRING_IDB,Database,IntAct_score,BioGRID_score,STRING_score,...,HPA_Subcellular_locationB,CORUM_complexesB,CORUM_subunit_IDsB,DisGeNet_disease_nameB,DisGeNet_disease_IDB,DisGeNet_classB,DisGeNet_geneB,DisGeNet_scoreB,DisProt_nameB,DisProt_IDB
0,O15260,P84085,NaN,NaN,9606.ENSP00000361057,9606.ENSP00000000233,S,NaN,NaN,0.909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O43731,P84085,NaN,NaN,9606.ENSP00000386918,9606.ENSP00000000233,S,NaN,NaN,0.917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P42771,P84085,NaN,NaN,9606.ENSP00000418915,9606.ENSP00000000233,S,NaN,NaN,0.606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A5PKW4,P84085,NaN,NaN,9606.ENSP00000020673,9606.ENSP00000000233,S,NaN,NaN,0.580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,O15155,P84085,NaN,NaN,9606.ENSP00000222547,9606.ENSP00000000233,S,NaN,NaN,0.921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create a NetworkX PPI graph

In [2]:
ppi = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI_Metadata_IDs.csv', encoding = 'utf-8', compression = 'gzip', low_memory = False, sep = '\t')
ppi2 = ppi.copy()

## Put the data from the dataframe into tuples and dictionaries so the nodes and edges have their corresponding attributes

### The nodes

In [3]:
# create 2 dataframes with the A and B identifiers and rename the columns
UA = ppi2[['UniprotAccessionA', 
           'BioGRID_IDA', 
           'STRING_IDA', 
           'Protein_nameA',  
           'Gene_namesA', 
           'GO_biological_processA', 
           'GO_cellular_componentA', 
           'GO_molecular_functionA', 
           'Reactome_IDA', 
           'Reactome_nameA', 
           'HPA_geneA', 
           'HPA_gene_descriptionA',
           'HPA_Subcellular_locationA', 
           'CORUM_complexesA', 
           'CORUM_subunit_IDsA', 
           'DisGeNet_disease_nameA', 
           'DisGeNet_disease_IDA', 
           'DisGeNet_classA',
           'DisGeNet_geneA', 
           'DisGeNet_scoreA', 
           'DisProt_nameA', 
           'DisProt_IDA']]
UA.rename(columns = {'UniprotAccessionA' : 'UniprotAccession',
                     'BioGRID_IDA' : 'BioGRID_ID', 
                     'STRING_IDA' : 'STRING_ID', 
                     'Protein_nameA' : 'Protein_name', 
                     'Gene_namesA' : 'Gene_names',     
                     'GO_biological_processA' : 'GO_biological_process', 
                     'GO_cellular_componentA' : 'GO_cellular_component', 
                     'GO_molecular_functionA' : 'GO_molecular_function', 
                     'Reactome_IDA' : 'Reactome_ID', 
                     'Reactome_nameA' : 'Reactome_name', 
                     'HPA_geneA' : 'HPA_gene', 
                     'HPA_gene_descriptionA' : 'HPA_gene_description', 
                     'HPA_Subcellular_locationA' : 'HPA_Subcellular_location',
                     'CORUM_complexesA' : 'CORUM_complexes', 
                     'CORUM_subunit_IDsA' : 'CORUM_subunit_IDs', 
                     'DisGeNet_disease_nameA' : 'DisGeNet_disease_name', 
                     'DisGeNet_disease_IDA' : 'DisGeNet_disease_ID', 
                     'DisGeNet_classA' : 'DisGeNet_class', 
                     'DisGeNet_geneA' : 'DisGeNet_gene', 
                     'DisGeNet_scoreA' : 'DisGeNet_score', 
                     'DisProt_nameA' : 'DisProt_name', 
                     'DisProt_IDA' : 'DisProt_ID'}, inplace=True)

UB = ppi2[['UniprotAccessionB', 
          'BioGRID_IDB', 
          'STRING_IDB', 
          'Protein_nameB', 
          'Gene_namesB', 
          'GO_biological_processB', 
          'GO_cellular_componentB', 
          'GO_molecular_functionB', 
          'Reactome_IDB', 
          'Reactome_nameB', 
          'HPA_geneB', 
          'HPA_gene_descriptionB', 
          'HPA_Subcellular_locationB', 
          'CORUM_complexesB', 
          'CORUM_subunit_IDsB', 
          'DisGeNet_disease_nameB', 
          'DisGeNet_disease_IDB', 
          'DisGeNet_classB',
          'DisGeNet_geneB', 
          'DisGeNet_scoreB', 
          'DisProt_nameB', 
          'DisProt_IDB']]
UB.rename(columns = {'UniprotAccessionB' : 'UniprotAccession',
                     'BioGRID_IDB' : 'BioGRID_ID', 
                     'STRING_IDB' : 'STRING_ID', 
                     'Protein_nameB' : 'Protein_name',  
                     'Gene_namesB' : 'Gene_names',     
                     'GO_biological_processB' : 'GO_biological_process', 
                     'GO_cellular_componentB' : 'GO_cellular_component', 
                     'GO_molecular_functionB' : 'GO_molecular_function', 
                     'Reactome_IDB' : 'Reactome_ID', 
                     'Reactome_nameB' : 'Reactome_name', 
                     'HPA_geneB' : 'HPA_gene', 
                     'HPA_gene_descriptionB' : 'HPA_gene_description', 
                     'HPA_Subcellular_locationB' : 'HPA_Subcellular_location', 
                     'CORUM_complexesB' : 'CORUM_complexes', 
                     'CORUM_subunit_IDsB' : 'CORUM_subunit_IDs', 
                     'DisGeNet_disease_nameB' : 'DisGeNet_disease_name', 
                     'DisGeNet_disease_IDB' : 'DisGeNet_disease_ID', 
                     'DisGeNet_classB' : 'DisGeNet_class', 
                     'DisGeNet_geneB' : 'DisGeNet_gene', 
                     'DisGeNet_scoreB' : 'DisGeNet_score', 
                     'DisProt_nameB' : 'DisProt_name', 
                     'DisProt_IDB' : 'DisProt_ID'}, inplace=True)


#add the two dataframes together and remove the duplicate rows
U = UA.append(UB)

C:\Anaconda\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [4]:
# Open the files that contain the Uniprot IDs and the STRING and BioGRID IDs
ub = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\UB_IDs.csv', sep = '\t')
us = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\US_IDs.csv', sep = '\t')

In [5]:
# Rename the columns to work with
ub.rename(columns = {'UniProtKB_ID' : 'UniprotAccession', 
                     'BioGRID_ID' : 'BioGRID1'}, inplace=True)
us.rename(columns = {'UniProtKB_ID' : 'UniprotAccession', 
                     'protein' : 'STRING'}, inplace=True)

#adjust the ub dataframe to get all the BioGRID IDs per UniProt ID
# Explode the cells to get each UniProt ID with a seperate BioGRID ID
ub = ub.assign(BioGRID = ub['BioGRID1'].str.split(',')).explode('BioGRID')

# Remove the other column and group the BioGRID IDs per UniProt ID and drop all the duplicates
ub = ub[['UniprotAccession', 'BioGRID']]
ub['BioGRID'] = ub.groupby(['UniprotAccession'])['BioGRID'].transform(lambda x: ', '.join(x))
ub.drop_duplicates(inplace = True)

In [6]:
# Merge the dataframes together to get all possible information
U = pd.merge(U, ub, left_on='UniprotAccession', right_on='UniprotAccession', how = 'left')
U = pd.merge(U, us, left_on='UniprotAccession', right_on='UniprotAccession', how = 'left')

# Retain only the necessary columns and rename the columns if need be
U1 = U[['UniprotAccession', 
        'BioGRID', 
        'STRING',        
        'Protein_name',  
        'Gene_names', 
        'GO_biological_process', 
        'GO_cellular_component', 
        'GO_molecular_function',  
        'Reactome_ID', 
        'Reactome_name', 
        'HPA_gene', 
        'HPA_gene_description', 
        'HPA_Subcellular_location', 
        'CORUM_complexes', 
        'CORUM_subunit_IDs', 
        'DisGeNet_disease_name', 
        'DisGeNet_disease_ID', 
        'DisGeNet_class', 
        'DisGeNet_gene', 
        'DisGeNet_score', 
        'DisProt_name', 
        'DisProt_ID']]
U1.rename(columns = {'BioGRID' : 'BioGRID_ID', 
                     'STRING' : 'STRING_ID'}, inplace=True)

# Remove all duplicates and print the amount of nodes the network will contain
U1.drop_duplicates(inplace = True)
uniprot_ids = U1.copy()
uniprot_ids.shape

<ipython-input-6-98f884d93601>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  U1.drop_duplicates(inplace = True)


(18683, 22)

In [7]:
# set the UniProt IDs as an index
uniprot_ids.set_index('UniprotAccession', inplace = True)

# Remove all duplicate terms in all the relevant columns
uniprot_ids = uniprot_ids.astype(str)
uniprot_ids['Reactome_ID'] = uniprot_ids['Reactome_ID'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['Reactome_name'] = uniprot_ids['Reactome_name'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['HPA_gene'] = uniprot_ids['HPA_gene'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['HPA_gene_description'] = uniprot_ids['HPA_gene_description'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['HPA_Subcellular_location'] = uniprot_ids['HPA_Subcellular_location'].str.split(',').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['CORUM_complexes'] = uniprot_ids['CORUM_complexes'].str.split(' / ').apply(lambda x: ' ; '.join(list(set(x))))
uniprot_ids['CORUM_subunit_IDs'] = uniprot_ids['CORUM_subunit_IDs'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisGeNet_disease_name'] = uniprot_ids['DisGeNet_disease_name'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisGeNet_disease_ID'] = uniprot_ids['DisGeNet_disease_ID'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisGeNet_class'] = uniprot_ids['DisGeNet_class'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisGeNet_gene'] = uniprot_ids['DisGeNet_gene'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisGeNet_score'] = uniprot_ids['DisGeNet_score'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisProt_name'] = uniprot_ids['DisProt_name'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))
uniprot_ids['DisProt_ID'] = uniprot_ids['DisProt_ID'].str.split(' / ').apply(lambda x: '; '.join(list(set(x))))

In [8]:
# Put all  the rows into dictionaries without the UniProt IDs
x = uniprot_ids.to_dict('records')

# Put the UniProt IDs in a list
y = list(uniprot_ids.index.values)

# Combine these two lists into a list of tuples for the network
node_attr = []
for i in range(len(y)):
    node_attr.append(tuple([y[i], x[i]]))
len(node_attr)

18683

### The edges

In [9]:
# Create a dataframe out of the columns with edge attributes and put the values per interaction in a dictionary
ppi1 = ppi2[['IntAct_score', 
             'BioGRID_score', 
             'STRING_score', 
             'Database', 
             'BioGRID_ontology_ID']]

x2 = ppi1.to_dict('records')

# Create two lists that each contain the interaction pairs (one of the pair in one list and the other in the other list)
u1 = list(ppi2.UniprotAccessionA.values)
u2 = list(ppi2.UniprotAccessionB.values)

# Combine these three lists into a list of tuples for the network
edge_attr = []
for i in range(len(x2)):
    edge_attr.append(tuple([u1[i], u2[i], x2[i]]))
len(edge_attr)

611761

## Create the network with the nodes and edges and their attributes

In [10]:
# Create empty graph
G = nx.Graph()

# Add the nodes
G.add_nodes_from(node_attr)

#Add the edges
G.add_edges_from(edge_attr)

In [11]:
G.number_of_nodes()

18683

In [12]:
G.number_of_edges()

543266

## Recombine all the information in a dataframe (interaction and analysis data)

In [13]:
#Get the interactions from the network
ppi = nx.to_pandas_edgelist(G)
ppi.rename(columns = {'source' : 'UniprotAccession_A', 
                      'target' : 'UniprotAccession_B'}, inplace = True)

# merge the uniprot IDs with the interactions
ppi = pd.merge(ppi, uniprot_ids, left_on = 'UniprotAccession_A', right_on = 'UniprotAccession', how = 'left')
ppi = pd.merge(ppi, uniprot_ids, left_on = 'UniprotAccession_B', right_on = 'UniprotAccession', how = 'left')

# Alter the suffixes added
ppi.columns = ppi.columns.str.replace(r'_x', '_A')
ppi.columns = ppi.columns.str.replace(r'_y', '_B')
ppi = ppi.loc[:,~ppi.columns.duplicated()]

ppi.shape

(543266, 49)

In [14]:
# Only retain the columns necessary
ppi_final = ppi[['UniprotAccession_A', 
                 'BioGRID_ID_A', 
                 'STRING_ID_A', 
                 'Protein_name_A',  
                 'Gene_names_A',
                 'UniprotAccession_B', 
                 'BioGRID_ID_B', 
                 'STRING_ID_B', 
                 'Protein_name_B',  
                 'Gene_names_B', 
                 'IntAct_score', 
                 'BioGRID_score', 
                 'STRING_score', 
                 'Database', 
                 'BioGRID_ontology_ID', 
                 'GO_biological_process_A', 
                 'GO_cellular_component_A', 
                 'GO_molecular_function_A', 
                 'Reactome_ID_A', 
                 'Reactome_name_A', 
                 'HPA_gene_A', 
                 'HPA_gene_description_A', 
                 'HPA_Subcellular_location_A', 
                 'CORUM_complexes_A', 
                 'CORUM_subunit_IDs_A', 
                 'DisGeNet_disease_name_A', 
                 'DisGeNet_disease_ID_A', 
                 'DisGeNet_class_A', 
                 'DisGeNet_gene_A', 
                 'DisGeNet_score_A', 
                 'DisProt_name_A', 
                 'DisProt_ID_A', 
                 'GO_biological_process_B', 
                 'GO_cellular_component_B', 
                 'GO_molecular_function_B', 
                 'Reactome_ID_B', 
                 'Reactome_name_B', 
                 'HPA_gene_B', 
                 'HPA_gene_description_B', 
                 'HPA_Subcellular_location_B', 
                 'CORUM_complexes_B', 
                 'CORUM_subunit_IDs_B', 
                 'DisGeNet_disease_name_B', 
                 'DisGeNet_disease_ID_B', 
                 'DisGeNet_class_B', 
                 'DisGeNet_gene_B', 
                 'DisGeNet_score_B', 
                 'DisProt_name_B', 
                 'DisProt_ID_B']]

In [15]:
ppi_final['Database'].value_counts()

S      494765
BS      16455
IBS     12819
IS       9997
I        8217
IB       1013
Name: Database, dtype: int64

In [16]:
#Save the dataframe both in csv and txt format and  save the uniprot_ids as well
ppi_final.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
ppi_final.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI.txt', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')